## 1. Permohonan aplikasi kartu kredit 
Bank komersial mendapatkan banyak form aplikasi untuk membuat kartu kredit. Permintaan kartu kredit ditolak dengan berbagai alasan, contohnya, jumlah hutang yang tinggi, pendapatan rendah, atau masalah lain pada laporan kredit. Memeriksa semua form aplikasi secara manual menghabiskan banyak waktu, dan manusia cenderung membuat kesalahan. Pada artikel ini saya akan mencoba membuat model machine learning untuk memprediksi kelayakan aplikasi kartu kedit secara otomatis

<p>Berikut merupakan dataset <a href="http://archive.ics.uci.edu/ml/datasets/credit+approval">"Credit Card Approval"</a> dari UCI Machine Learning Repository.

In [40]:
# Import pandas
import pandas as pd
# Load dataset
cc_apps = pd.read_csv("datasets/cc_approvals.data", header=None)
cc_apps.head()

## 2. Aplikasi kartu kredit
Semua fitur pada dataset ini tidak diperlihatkan untuk menjaga privasi, namun <a href="http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html">blog ini</a> memberikan penjelasan yang cukup baik mengenai fitur-fitur yang mungkin dapat digunakan. Fitur-fitur yang biasa dadapati pada aplikasi kartu kredit adalah <code>Gender</code>, <code>Age</code>, <code>Debt</code>, <code>Married</code>, <code>BankCustomer</code>, <code>EducationLevel</code>, <code>Ethnicity</code>, <code>YearsEmployed</code>, <code>PriorDefault</code>, <code>Employed</code>, <code>CreditScore</code>, <code>DriversLicense</code>, <code>Citizen</code>, <code>ZipCode</code>, <code>Income</code> dan <code>ApprovalStatus</code>. Saat ini dataset ini hanya kumpulan fitur numerik dan non-numerikal, Masalah ini dapat diperbaiki dengan preprocessing, namun sebelum melakukan preprocessing, sebaiknya kita harus memeriksa semua masalah yang mungkin ada pada dataset ini yang perlu diperbaiki.

In [42]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print('\n')

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print('\n')
print(cc_apps.tail(17))

# Inspect missing values in the dataset
# ... YOUR CODE FOR TASK 2 ...

               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64

## 3. Membagi data menjadi train dan test set
Sekarang, saya akan membagi data menjadi 2 set yaitu train set dan test set. Untuk mempersiapkan data untuk dua tahapan lain dari proses machine learning yaitu training dan testing. Sebaiknya, tidak ada informasi pada test set yang digunakan untuk membantu proses preprocessing pada train set. maka dari itu pertama saya akan membagi data terlebih dahulu kemudian membaginya. Selain itu fitur-fitur seperti <code>DriversLicense</code> dan <code>ZipCode</code> tidak terlalu penting untuk digunakan pada proses training dibandingkan dengan fitur-fitur yang lain untuk memprediksi kelayakan aplikasi kartu kredit

In [44]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop fitur 11 dan 13
cc_apps = cc_apps.drop([11,13], axis=1)

# Split data menjadi train dan test set
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

## 4. Mengatasi missing values (part i)
Setelah data dibagi menjadi train dan test set. kita bisa mulai mengatasi masalah yang kita temui ketika memeriksa dataframe:
<ul>
<li>Dataset yang digunakan saat ini memiliki data numerik dan non-numerik(<code>float64</code>, <code>int64</code> dan <code>object</code>)</li>
<li>Dataset ini juga memiliki berbagai rentang nilai. Ada fitur memiliki rentang nilai 0-28, ada juga yang memiliki rentang 2-67, bahkan 1017-100000.</li>
<li>Yang terakhir pada dataset beberapa baris pada setiap kolom tidak memiliki nilai (missing values). Missing values pada dataset ditandai dengan simbol ?</li>
</ul>

In [46]:
# Import numpy
import numpy as np 

# Mengganti simbol ? dengan NaN pada train dan test set 
cc_apps_train = cc_apps_train.replace('?',np.NaN)
cc_apps_test = cc_apps_test.replace('?',np.NaN)

## 5. Mengatasi missing values (part ii)
Semua ? sudah diganti dengan NaN. Ini bertujuan untuk membantu kita untuk pada tahapan selanjutnya untuk mengatasi missing value, yaitu dengan strategi mean imputation 

In [48]:
# Imputasi missing values dengan mean imputation
cc_apps_train.fillna(cc_apps_train.mean(), inplace=True)
cc_apps_test.fillna(cc_apps_test.mean(), inplace=True)

# Tampilkan jumlah nilai NaN pada dataset untuk memverifikasi
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

## 6. Mengatasi missing values (part iii)
Kita telah berhasi mengatasi missing values yang ada pada kolom yang memiliki nilai numerikas. namun masih banyak missing values yang perlu di imputasi pada kolom 0, 1, 3, 4, 5, 6, dan 13. Semua kolom ini memiliki nilai non-numerik, maka dari itu strategi mean imputation tidak akan berhasil jika digunakan pada kolom tersebut. Imputasi pada kolom-kolom ini akan dilakukan dengan menggunakan nilai yang sering muncul pada kolom tersebut. Metode ini secara umum baik digunakan ketika melakukan imputasi untuk data kategorikal. 

In [50]:
# Iterasi untuk setiap kolom pada cc_apps_train 
for col in cc_apps_train.columns:
    # Jika tipe data kolom tersebut object
    if cc_apps_train[col].dtypes == 'object':
        # Imputasi dengan nilai yang paling sering muncul
        cc_apps_train = cc_apps_train.fillna(cc_apps_train[col].value_counts().index[0])
        cc_apps_test = cc_apps_test.fillna(cc_apps_train[col].value_counts().index[0])

# Hitung jumlah nilai NaN yang ada pada dataset 
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


## 7. Data preprocessing (part i)
Semua missing values telah berhasil diatasi. 
Masih ada beberapa langkah data preprocessing yang diperlukan sebelum mulai membangun model machine learning. Saya akan membagi tahapan preprocessing ini menjadi dua tahapan:
<ol>
<li>Ubah data non-numerik menjadi numerik.</li>
<li>Manipulasi data sehingga semua nilai fitur memiliki rentang nilai yang sama</li>
</ol>
Pertama, saya akan mengubah semua nilai non-numerik menjadi numerik. Hal ini penting dilakukan untuk mendapatkan proses komputasi yang lebih cepat, selain itu banyak model machine learning (terutama yang ada pada library scikit-learn) mengharuskan data dengan format numerik. Hal ini akan dilakukan dengan menggunakan method <code>get_dummies()</code> dari library pandas.


In [52]:
pd.get_dummies(cc_apps_train)

,2,7,10,14,0_a,0_b,1_13.75,1_15.83,1_15.92,1_16.00,...,6_z,8_f,8_t,9_f,9_t,12_g,12_p,12_s,15_+,15_-
382,2.500,4.500,0,456,1,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
137,2.750,4.250,6,0,0,1,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
346,1.500,0.250,0,122,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
326,1.085,0.040,0,179,0,1,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
33,5.125,5.000,0,4000,1,0,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
426,2.040,0.250,0,50,1,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,1
477,2.500,10.000,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,1
9,4.915,3.165,0,1442,0,1,0,0,0,0,...,0,0,1,1,0,1,0,0,1,0
22,8.000,7.875,6,1260,1,0,0,0,0,0,...,0,0,1,0,1,1,0,0,1,0
300,2.000,6.500,1,10,1,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,1


In [53]:
# Ubah fitur pada train dan test set secara terpisah
cc_apps_train = pd.get_dummies(cc_apps_train)
cc_apps_test = pd.get_dummies(cc_apps_test)

# Reindex kolom-kolom yang ada pada test set agar sejajar dengan train set
cc_apps_test = cc_apps_test.reindex(columns=cc_apps_train.columns, fill_value=0)

In [54]:
cc_apps_train.iloc[:, :-1].values, cc_apps_train.iloc[:, [-1]].values

(array([[2.5  , 4.5  , 0.   , ..., 0.   , 0.   , 0.   ],
        [2.75 , 4.25 , 6.   , ..., 0.   , 0.   , 1.   ],
        [1.5  , 0.25 , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 1.   , 0.   , 1.   ],
        [0.   , 0.   , 4.   , ..., 0.   , 0.   , 0.   ],
        [5.   , 0.375, 2.   , ..., 0.   , 0.   , 0.   ]]),
 array([[1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
      

## 8. Data preprocessing (part ii)
Proses preprocessing terakhir adalah mengubah rentang nilai data (Rescaling). untuk menjelaskan proses rescaling saya gunakan contoh fitur <code>CreditScore</code>. Credit score seseorang adalah sebuah nilai kelayakan seseorang untuk memiliki kredit berdasarkan <i>credit history</i> mereka. Semakin tinggi credit score seseorang maka secara finansial mereka akan semakin dipercayai untuk memiliki kredit. Jadi, saya akan melakukan rescaling pada fitur <code>CreditScore</code> dan fitur-fitur lain sehingga memiliki rentang nilai dari 0-1 dimana 0 adalah nilai terendah dan 1 adalah nilai tertinggi

In [56]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Pisahkan fitur dan label ke variabel yang berbeda
X_train, y_train = cc_apps_train.iloc[:, :-1].values, cc_apps_train.iloc[:, [-1]].values
X_test, y_test = cc_apps_test.iloc[:, :-1].values, cc_apps_test.iloc[:, [-1]].values

# Gunakan MinMaxScaler untuk rescaling X_train dan X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

## 9. Fitting model logistic regression dengan train set
Pada dasarnya, memprediksi jika aplikasi kartu kredit akan diterima atau tidak termasuk dalam jenis kasus <a href="https://en.wikipedia.org/wiki/Statistical_classification">klasifikasi</a>. Menurut UCI, dataset yang saya gunakan saat ini memiliki lebih banyak aplikasi yang berstatus "Denied" daripada yang berstatus "Approved". Dari 690 baris ada 383(55.5%) aplikasi dengan status "Denied" dan 307(44.5%) dengan status "Approved". Model machine learning yang baik seharusnya dapat secara akurat memprediksi status aplikasi sesuai dengan statistik tersebut.

Model machine learning mana yang sebaiknya kita pilih?. Karena semua kolom yang ada pada dataset ini memiliki korelasi yaitu sebagai nilai ukur layak atau tidaknya sebuah apliksi kartu kredit, saya akan memilih menggunakan <i>Logistic Regression</i> karena model tersebut biasanya memberikan hasil yang baik pada kasus serupa. 

In [58]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Inisiasi LogisticRegression classifier dengan menggunakan nilai parameter default 
logreg = LogisticRegression()

# fit LogisticRegression ke train set
logreg.fit(rescaledX_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## 10. Membuat prediksi dan mengevaluasi model machine learning
Selanjutnya adalah mengukur seberapa baik performa model machine learning yang telah dibuat.

Saya akan mengevaluasi model ini dengan test set untuk mengukur <a href="https://developers.google.com/machine-learning/crash-course/classification/accuracy">classification accuracy</a>. Tapi sebelum itu saya juga akan melihat <a href="http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/">confusion matrix</a> dari model yang telah dibuat. Pada kasus memprediksi kelayakan aplikasi kartu kredit, penting untuk melihat apakah model yang telah dibuat telah berhasil memprediksi status "Approved" dan "Denied" yang setara dengan frekuensi label pada dataset awal. Jika hasilnya tidak sesuai dengan aspek tersebut, maka mungkin saja model yang dibuat nantinya akan memberikan status "Approved" kepada aplikasi yang seharusnya diberikan status denied. Confusion Matrix akan membantu kita untuk memperlihatkan kesesuaian aspek tersebutk.

In [60]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Gunakan logistic regression untuk memprediksi semua data pada test set dan simpan pada variable y_pred
y_pred = logreg.predict(rescaledX_test)

# Print nilai akurasi dari model logistic regression
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))

# Print confusion matrix dari model logistic regression
confusion_matrix(y_test,y_pred)

Accuracy of logistic regression classifier:  1.0


array([[103,   0],
       [  0, 125]])

## 11. Grid searching dan meningkatkan performa model machine learning
Hasil prediksi model cukup baik, bahkan dapat mencapai nilai akurasi 100%.

Pada confusion matrix, element pertama dari baris pertama confusion matrix menunjukan "true negatives" artinya jumlah aplikasi yang ditolak oleh model dan memang seharusnya ditolak. element terakhir dari baris kedua confusion matrix menunjukan "true positives" artinya jumlah aplikasi yang ditolak oleh model dan memang seharusnya ditolak.

Model machine learning yang dihasilkan sangat baik, namun jika model machine learning mendapatkan performa yang tidak sesuai harapan hal yang bisa dilakukan adalah dengan melakukan grid search pada parameter model yang digunakan untuk mencari parameter yang sesuai agar model mendapatkan kemampuan prediksi yang baik. 

Implementasi logistic regression terdiri dari beberapa hyperparameter namun saya akan melakukan grid search pada dua hyperparameter berikut

<ul>
<li>tol</li>
<li>max_iter</li>
</ul>


In [62]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Tentukan nilai-nilai yang akan digunakan untuk tol, dan max_iter
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are the corresponding values
# Buat dictionary dimana tol dan max_iter adalah key dan value-nya adalah list dari nilai-nilai yang telah ditentukan sebelumnya pada masing-masing key
param_grid = dict(tol=tol, max_iter=max_iter)

## 12. Menentukan model dengan performa yang paling baik

Grid dari nilai hyperparameter telah ditentukan dan telah dibuat dalam bentuk dictionary yang mana akan digunakan <code>GridSearchCV()</code> sebagai salah satu dari parameterny. Sekarang, mulai grid search untuk melihat nilai mana yang memberikan performa terbaik.

<ul>
<li>Pertama saya akan menginisiasi <code>GridSearchCV()</code> dengan model logistic regression sebelumnya dengan data yang dimiliki. </li>
<li>Kemudia yang terakhir menyimpan skor terbaik dan nilai parameter yang digunakan</li>
</ul>

In [64]:
# Inisiasi GridSearchCV dengan semua parameter yang diperlukan
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Fit grid_model ke data
grid_model_result = grid_model.fit(rescaledX_train, y_train)

# Tampilkan hasil
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

# Gunakan model terbaik dan evaluasi pada test set
best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classifier: ", best_model.score(rescaledX_test,y_test))

Best: 1.000000 using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  1.0
